## Merge Root Files

In [31]:
import uproot
import os
import numpy as np

def merge_files(directory, starts_with, output_name):

    file_list = []

    for file in os.listdir(directory):
        if file.endswith('.root') and not file.startswith('merge') and not file.startswith(output_name):
            if not starts_with == '' and file.startswith(starts_with):
                    file_list.append(os.path.join(directory, file))
            else:
                file_list.append(os.path.join(directory, file))
    
    merged_file = os.path.join(directory, output_name)
    
    counter = 0
    while True:
        if not os.path.exists(f"{merged_file}{counter}.root"):
            merged_file = f"{merged_file}{counter}.root"
            break
        counter = counter + 1

    with uproot.recreate(merged_file) as f_out:
        data_dict = {}  # Dictionary to store merged data temporarily
        
        for file in file_list:
            
            with uproot.open(file) as f_in:
                for key in f_in.keys():
                    obj = f_in[key]
                    
                    if isinstance(obj, uproot.TTree):
                        new_data = obj.arrays(library="np")
                        
                        # Extract base key name (ignore cycle numbers)
                        base_key = key.split(';')[0]

                        # If base_key is already in data_dict, concatenate data
                        if base_key in data_dict:
                            existing_data = data_dict[base_key]
                            combined_data = {k: np.concatenate([existing_data[k], new_data[k]]) for k in new_data.keys() if k in existing_data}
                            # Update with the combined data
                            data_dict[base_key] = {**existing_data, **combined_data}
                        else:
                            # If base_key is not in data_dict, add new data
                            data_dict[base_key] = new_data

        for key, data in data_dict.items():
            f_out[key] = data


    print("Merged files into", merged_file)

In [ ]:
directory = 'BUILD/ROOT'
starts_with = 'root'
output_name = 'merge'

merge_files(directory, starts_with, output_name)

## Root to Dataframe

In [1]:
import uproot
import os
import numpy as np
import dask.array as da
import dask.dataframe as dd

def root_to_dask(directory, root_name_starts, tree_name, x_branch, y_branch, decimal_places):
    
    file_name = os.path.join(directory, root_name_starts + ".root")

    with uproot.open(file_name) as root_file:
        tree = root_file[tree_name]
        if tree is None:
            print(f"Tree '{tree_name}' not found in {file_name}")
            return

        x_values = tree[x_branch].array(library="np") if x_branch in tree else None
        y_values = tree[y_branch].array(library="np") if y_branch in tree else None

        if x_values is not None:
            x_values = np.round(x_values, decimal_places)
        if y_values is not None:
            y_values = np.round(y_values, decimal_places)

        if x_values is None or y_values is None:
            print(f"Could not retrieve data for branches {x_branch} or {y_branch}")
            return

        x_dask_array = da.from_array(x_values, chunks="auto")
        y_dask_array = da.from_array(y_values, chunks="auto")

        dask_df = dd.from_dask_array(da.stack([x_dask_array, y_dask_array], axis=1), columns=[x_branch, y_branch])
        
        return dask_df

In [11]:
directory = 'RESULTS/'
root_name_starts = "80kev_500M"

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"

tree_name = "G4_PCM"
x_branch  = "PositionX"
y_branch  = "PositionY"

z_branch  = ""

decimal_places = 2

dataframe = root_to_dask(directory, root_name_starts, tree_name, x_branch, y_branch, decimal_places)

x_branch = "X_axis"
y_branch = 'Y_axis'

x_branch = 'PositionX'
y_branch = 'PositionY'

x_data = dataframe[x_branch].to_dask_array(lengths=True)
y_data = dataframe[y_branch].to_dask_array(lengths=True)

## Dask Dataframe to Heatmap

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import dask.dataframe as dd

def heatmap_array_dask(x_data, y_data, size, num, save_as):

    set_bins = np.arange(-size, size + 1, size/num)
    # heatmap, x_edges, y_edges = np.histogram2d(x_data, y_data, bins = [set_bins, set_bins])
    heatmap, x_edges, y_edges = da.histogram2d(x_data, y_data, bins = [set_bins, set_bins])
    heatmap = heatmap.T

    heatmap = heatmap.compute()  
    x_edges = x_edges.compute()  
    y_edges = y_edges.compute()

    normal_map = 1 - heatmap / np.max(heatmap)

    columns = heatmap.shape[1]
    column_i = int(columns * 0.1)
    column_j = int(columns * 0.9)

    rows = heatmap.shape[0]
    row_i = int(rows * 0.01)
    row_j = int(rows * 0.1)

    maxi = np.max(normal_map[row_i:row_j, column_i:column_j])
    maxi = maxi * 1.15
    # print('altura de ruido:', round(maxi, 4))

    normal_map[normal_map < maxi] = 0

    # plt.figure(figsize=(10, 4))
    # plt.subplot(1, 2, 1)
    # plt.imshow(normal_map, cmap='gray', extent = [x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    # plt.subplot(1, 2, 2)
    # plt.plot(normal_map[:][110])
    
    plt.imshow(normal_map, cmap='gray', extent = [x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    if save_as != '': plt.savefig('Results/' + save_as + '.jpg', dpi = 900)

    return normal_map, x_edges, y_edges

In [ ]:
size = 100
bins = 30

save_as = ''

htmp_array, xlim, ylim = heatmap_array_dask(x_data, y_data, size, bins, save_as)

## Eliminate Noise by Fourier Transform

In [42]:
import numpy as np
from scipy.fft import fft2, fftshift, ifft2
import matplotlib.pyplot as plt
from scipy import signal

def Denoise(htmp_array, hann, alpha):

    image = htmp_array

    fft_image = fft2(image)
    fft_image = fftshift(fft_image)

    rows, cols = image.shape

    hann = False
    if hann == True:
    
        l = rows * alpha
        a = np.hanning(l)
        b = np.hanning(l)

        padding_size = rows - len(a)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        a = np.pad(a, (left_padding, right_padding), mode='constant')

        padding_size = cols - len(b)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        b = np.pad(b, (left_padding, right_padding), mode='constant')

        window = np.outer(a, b)

    else:

        a = signal.windows.tukey(rows, alpha)
        b = signal.windows.tukey(rows, alpha)
        window = np.outer(a, b)

    plt.figure(figsize=(10, 2))
    plt.subplot(1, 3, 2)
    plt.plot(a)

    fft_image_2 = fft_image * (window)

    plt.subplot(1, 3, 3)
    plt.plot(np.abs((fft_image_2[:][rows//2])))

    fft_image = fftshift(fft_image_2)
    fft_image = (ifft2(fft_image))
    fft_image = (np.abs(fft_image))

    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title('Filtered Image')
    plt.imshow(fft_image, cmap='gray')
    # plt.savefig('Results/four1.jpg', dpi = 300)
    plt.show()

    print(fft_image.shape)
    plt.plot(fft_image[:][60])

    return fft_image

In [ ]:
hann = False
alpha = .4
array = htmp_array
fft_image = Denoise(array, hann, alpha)

## Denoise with Skimage.Denoise_Bilateral

In [ ]:
from skimage.restoration import denoise_bilateral
import matplotlib.pyplot as plt

image = htmp_array
denoised_image = denoise_bilateral(image, sigma_color = 0.03, sigma_spatial = 1, channel_axis = None)

# Display the original and denoised images
# fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# ax[0].imshow(image, cmap='gray')
# ax[0].set_title('Original Image')
# ax[0].axis('off')

# ax[1].imshow(denoised_image, cmap='gray')
# ax[1].set_title('Denoised Image')
# ax[1].axis('off')

plt.imshow(denoised_image, cmap='gray')
plt.savefig('denoised_image.png', dpi = 900)
plt.show()

## Save Plotly Heatmap

In [10]:
import plotly.graph_objects as go
import plotly.io as pio

def heatmap2(array, xlim, ylim, title, x_label, y_label, width, height, save_as):

    fig = go.Figure(go.Heatmap(z = array, x = xlim, y = ylim,
                                colorscale = [[0, 'black'], [1, 'white']], 
                                colorbar = dict(title = "Density", tickfont = dict(family = 'Merriweather', size = 16, color = 'Black'))))
    
    font_family = 'Merriweather'
    font_small  = 16
    font_medium = 20
    font_large  = 18
    
    fig.update_layout(
                    title = dict(text = title, font = dict(family = font_family, size = font_large, color = "Black"), 
                                 x = 0.51, y = 0.93, yanchor = 'middle', xanchor = 'center'),
                    xaxis_title = dict(text = x_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    yaxis_title = dict(text = y_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    xaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 25),
                    yaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 10),
                    width  = width,
                    height = height,
                    margin = dict(l = 105, r = 90, t = 90, b = 90)
    )
   
    if save_as != '': pio.write_image(fig, 'Results/' + save_as + '.jpg', width = width, height = height, scale = 5)
    fig.show()

In [ ]:
array = htmp_array
# array = denoised_image
# array = fft_image

title   = r"$ \large{ \text{Healthy Bone, 80keV, 250M Beams,} } \ \normalsize{ \theta = 0° } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

save_as = ''

heatmap2(array, xlim, ylim, title, x_label, y_label, width, height, save_as)

## Calculate Fixed CNR 

In [13]:
from PIL import Image
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

def CNR(image_path):
    image = Image.open(image_path)
    cropped_image = image.crop((520, 450, image.width - 580, image.width - 440))
    data = np.array(cropped_image)

    plt.imshow(data)

    signal_avg = 0
    background_avg = 0
    background_std = 0

    x1_signal = 1160
    y1_signal = 1130
    x2_signal = 1740
    y2_signal = 1980

    plt.gca().add_patch(plt.Rectangle((x1_signal, y1_signal), x2_signal - x1_signal, y2_signal - y1_signal, 
                                    linewidth=2, edgecolor='yellow', facecolor='none'))


    if x2_signal > x1_signal:
        if y2_signal > y1_signal:
            signal = data[round(y1_signal):round(y2_signal), round(x1_signal):round(x2_signal)]
        else:
            signal = data[round(y2_signal):round(y1_signal), round(x1_signal):round(x2_signal)]
    else:
        if y2_signal > y1_signal:
            signal = data[round(y1_signal):round(y2_signal), round(x2_signal):round(x1_signal)]
        else:
            signal = data[round(y2_signal):round(y1_signal), round(x2_signal):round(x1_signal)]

    signal_avg = np.average(signal)
    print("Signal avg: "+str(signal_avg))

    x1_background = 1790
    y1_background = 1130
    x2_background = 1940
    y2_background = 1980

    plt.gca().add_patch(plt.Rectangle((x1_background, y1_background), x2_background - x1_background, y2_background - y1_background, 
                                    linewidth=2, edgecolor='red', facecolor='none'))


    if x2_background > x1_background:
        if y2_background > y1_background:
            background = data[round(y1_background):round(y2_background), round(x1_background):round(x2_background)]
        else:
            background = data[round(y2_background):round(y1_background), round(x1_background):round(x2_background)]
    else:
        if y2_background > y1_background:
            background = data[round(y1_background):round(y2_background), round(x2_background):round(x1_background)]
        else:
            background = data[round(y2_background):round(y1_background), round(x2_background):round(x1_background)]

    background_avg = np.average(background)
    background_std = np.std(background)

    print("Background avg: "+str(background_avg))
    print("Background std dev: "+str(background_std))

    cnr = (signal_avg-background_avg)/background_std
    print("CNR: ", round(cnr, 1))

In [ ]:
image_path = "RESULTS/" + "" + "prueba2" + ".jpg"

CNR(image_path)